> The following notes were taken during lecture 04/10/2017

In review, consider the following energy equation as it reacts to $\alpha$

$$E(Y,Z) = ||Y-WZ||^2 + \alpha \sum_i |Z_i|$$

In effect, trying to recreate a datapoint as a weighted sum of vectors.

The inference algorithm tries to find

$$Z^* = argmin_z\ E(Y,Z)$$

There is an algorithm which accomplishes this call (isTA) iterative shrinkage and thresholding alg.

$$z(T+1) = shrink \big[ W^TY+ \eta S Z_{(k)}$$

S is matrix computed from W with constant (later)
Shrink is point-wise non-linearity /---/

Why not PCA?  Z may not have same same D as Y.  **This is main advantage of sparce coding**.

To learn W matrix, apply (effectively) SGD:

$$ W \leftarrow W + \eta (Y -WZ) Z^T $$

$\eta$ is stepsize

Optimize W s.t. minimize reconstruction error using $argmin$.

Need to be careful, there is L1 criterion on Z.  Normalize columns of W so it has all unique norms.  This is effectively projecting onto hypersphere.

$$ W \leftarrow norm\_columns(W) $$

These two functions are the beating heart of sparse coding.

> Shoutout to Julien Mairal, SPAMS http://spams-devel.gforge.inria.fr/

Sparse coding learns a basis.  Looking at MNIST example, we are in fact computing shadow-looking basis images in columns of the W matrix.  Cool!

> Kavukcuoglu, Ranzato, LeCun, rejected by every conference, 2008-2009

Generative model for sparse encoder.  We are training it to identity function ONLY ON SAMPLE WE PROVIDE, not general image.  We achieve this with sparsity factor.

> Z can be optimized using "Target Prop"

## Energy function for inference

arXiv:1010.3467 Predictive Sparse Decomposition

## Better idea: FISTA

Gregor & LeCun, ICML, 2010, Bronstein et al. ICML, 2012, Rolfe & LeCun ICLR 2013

$$
Z(t+1) =
    \texttt{Shrinkage}_{\lambda/L} \big[
        Z(t) - \frac{1}{L}W_d^T(W_dZ(t)-Y)
    \big]
$$

$$
Z(t+1) =
    \texttt{Shrinkage}_{\lambda/L} \big[
        W_e^TY + SZ(t)\big]; W_e = \frac{1}{L}W_d ...
$$
    


Basically we are using machine learning to improve inference.

## Learning algorithms to train iterative alg for fast inference procedure

Same as before, but now encoder is recurrent network.

In experiments, LISTA outperforms FISTA by lots!  This idea has really taken off in the last year

Discriminative REcurrent Sparse Auto-Encoder (DrSAE, Rolfe & LeCun ICLR 2013)

- Rectified linear units
- Classification loss: cross-entropy
- Reconstruction loss: squared error
- Sparsity penalty: L1 norm of hidden layer
- Rows of $W_d$ and columns of $W_e$ constrained in unit sphere

> *Q: Is one-shot learning special case of sparse learning?*

# Convolutional Sparse Coding

Regular sparse coding:

Convolutional S.C.:
$$
E(Y,Z) =
    ||Y - \sum_k W_k * Z_k ...
$$

Sometimes called "deconvolutional network".

Advantage: in patch learning, we need copy of every filter (oriented edges) at every position in patch.  With convolution operator, we can learn more complex things!

Filters and Basis functions obtained with 1,2,4,8,16,32,64 filters.

Yann idea!

1. train first layer using PSD
2. Discard reconstruction, use encoder and absolute value as feature extractor (treat L1 norm as ReLU)
3. train the second layer using PSD
4. ... continue as deep as needed

This helps if low samples or low labels.  Unsupervised pre-training!  Very cool!  Kavukcuoglu et al. NIPS 2010.

# Learning Invariant Features with L2 Group Sparsity

- Number of feature ...
- Pools tend to regroup similar features

$$
E(Y,Z) = 
    ||Y-W_d||^2 + Z-g_e(W_e,Y)||^2 + \sum_j\sqrt{\sum_{k \in P_j} Z_k^2}
$$

I want to minimize sum of these guys, during training, minimize number of non-zero components in vector.  Activate smallest number of units necessary!  Pooling over a region, I don't care how many are active, just want minimal number of active units.

### Groups are local in a 2D Topographic Map

Convolution kernels that result from this training have "fingerprint" quality.  Within group, as many as possible should fire.  Between groups, as few should fire as possible.  Like in the brain, neighboring neurons detect similar features.

Simple algorithms, simple constraints result in beautiful emergence of patterns appearing in nature (human brain).

# Exploiting Temporal Consistency

> Scattering transform, Stephane Mallat, Joan Bruna

> Mark Tygert has a publication on Nerual Computation (math just. for complex number)

> *Q:Does PyTorch support complex numbers?  Why not?*

## Invariant Features through Temporal Consistency

In shape example, we have 2 explanitory factors (shape, size).  We want a network to learn there are two independent explanatory factors.

Karol Gregor et al. (at NYU, now at DeepMind)

Mapping Units, (Hinton 1981), capsules (Hinton 2011)

If you learn things independently, you can choose which to care about!  Think grabbing an object.  We don't care what it is, we just want to grasp it!

How could we do this?

One idea is temp. constistency.  Think cue that image is same target in different positions.

Assumption: between two frames, shape won't change much, position won't change much.  So given shape is similar, we can exploit consistency:

Idea Karol had: system reconstructs successive frames that are observed, use two paths:

#### Decoder

1. Represent identity (same for all three frames)
2. Represent position (different for all three frames)

Combine multiplicitivly (term-by-term).  Plug that in to reconstruction network to recreate image.

Training: show 3 successive frames.  Through sparse-coding/inference ...

Why good idea?  You might imagine role of edger is "edge exists, not sure where...".  Role of placer is "something is here, not sure what".  What*where is edge/orientation!

#### Encoder

Same thing, but backwards.

## Sparse Auto-Encoder with "Slow Feature" Penalty

Goroshin et al. ICCV 2015 ArXiv: 1412.6056

# Integrating Supervised & Unsupervised Learning with Stacked What-Where Auto-Encoders

*a.k.a. ladder networks (hey, we tried this!)*

A ladder network is a FF-network paired with a reverse network of the same topology.  There are typically some lateral connections (indices) as well.

## Supervised and Unsupervised in One Learning Rule?

Problem: need a one-to-many mapping (not a function!)


Idea: keep the complementary information around

- So that the generative path is a function

What-Where Auto-Encoder

- [Ranzato 2007], [Gregor 2011], Hinton's Capsules

- Very old ideas by Hinton & Zemel, von der Malsburg and others about separating identity from instantiation parameters.

- Propagate "where" as indices of maxes.  "what" is the invariant code learned by feed-forward layer.

### A funny kind of pooling/unpooling

To make this all work with SGD we make pooling/unpooling smooth:

$$
m_k =
    \sum_{N_K}z(f,x,u)
        \frac
            {e^{ \beta z (f,x,y) }}
            {\sum_{N_k}e^{\beta z(f',x',y')}}
\approx {max}_{N_k} z(f,x,y)
$$

SWWAE on adversarial images